Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

T.b.d.


In [83]:
import logging
import random
import copy

In [84]:
N_HEAPS = 3
N_GAMES = 100

In [85]:
logging.basicConfig(level=logging.DEBUG)

In [86]:
class Player:
    def __init__(self, name:str, strategy, *strategy_args):
        self._strategy = strategy
        self._strategy_args = strategy_args
        self._name = name
        self._loser = False
        self._n_plies = 0
    
    def ply(self, state):
        self._n_plies += 1
        return self._strategy(self, state, *self._strategy_args)
    
    @property
    def loser(self):
        return self._loser
    
    @loser.setter
    def loser(self, val):
        self._loser = val
    
    @property
    def name(self):
        return self._name
    
    @property
    def n_plies(self):
        return self._n_plies
    
    @n_plies.setter
    def n_plies(self, val):
        self._n_plies = val
    
    def flush_parameters(self):
        self._n_plies = 0
        self._loser = False

class Nim:
    def __init__(self, num_rows: int=None, rows:list=None, k: int = None) -> None:
        if num_rows is not None:
            self._rows = [i*2 + 1 for i in range(num_rows)]
        else:
            self._rows = rows
        self._k = k

    def nimming(self, row: int, num_objects: int, player:Player) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            player.loser = True
            #logging.info("Yeuch")
    
    @property
    def rows(self):
        return self._rows
    
    def copy(self):
        return copy.deepcopy(self)


def play(A:Player, B:Player, state:Nim) -> Player:
    logging.debug(f"{state.rows} : initial state")
    while not (A.loser or B.loser):
        A.ply(state)
        logging.debug(f"{state.rows} : state after {A.name} ply")
        if not A.loser:
            B.ply(state)
            logging.debug(f"{state.rows} : state after {B.name} ply")
    if A.loser:
        return B
    elif B.loser:
        return A

def match(A:Player, B:Player, state:Nim, n_games:int=1):
    winners = list()
    for i in range(n_games):
        initial_state = state.copy()
        A.flush_parameters()
        B.flush_parameters()
        w = play(A, B, initial_state)
        winners.append((w.name, w.n_plies))
    return winners

def print_match_result(A:Player, B:Player, res:list):
    n_A_win = 0
    n_games = len(res)
    for i in range(n_games):
        if res[i][0] == A.name:
            n_A_win += 1
    print(f"{A.name} won {n_A_win} times\n{B.name} won {n_games - n_A_win} times") 
    
def random_strategy(player:Player, heaps:Nim, decrement:bool=False):
    non_zero_heaps_idxs = [i for i, v in enumerate(heaps.rows) if v > 0] # choose a random non-zero heap
    idx_heap = random.choice(non_zero_heaps_idxs)
    if decrement:
        quantity = 1
    else:
        quantity = random.randint(1, heaps.rows[idx_heap]) # decrease it of a random quantity
    heaps.nimming(idx_heap, quantity, player)


## Task3.1 Nim-Sum

In [87]:
def nim_sum(l:list):
    sum = 0
    for _, v in enumerate(l):
        sum ^= v
    return sum

def nim_sum_strategy(player:Player, heaps: Nim):
    if sum(heaps.rows) == 0:
        raise Exception("There is no heap left!")
        
    n_heaps = len(heaps.rows)
    n_heaps_to_zero = len([i for i, h in enumerate(heaps.rows) if h == 0])
    n_heaps_to_one = len([i for i, h in enumerate(heaps.rows) if h == 1])
    n_heaps_greater_than_zero = n_heaps - n_heaps_to_zero
    n_heaps_greater_than_one = n_heaps_greater_than_zero - n_heaps_to_one

    # manage particular situations
    if n_heaps_greater_than_zero % 2 == 0 and n_heaps_greater_than_one == 1: #[1, a, 1, 1, 0, 0], a > 1
        # take all objects from the heap with more than 1 object
        heaps.nimming(heaps.rows.index(max(heaps.rows)), max(heaps.rows), player)
    elif n_heaps_greater_than_zero % 2 == 1 and n_heaps_greater_than_one == 1: #[1, a, 1, 0, 0], a > 1
        # take all objects but 1 from the heap with more than 1 object
        heaps.nimming(heaps.rows.index(max(heaps.rows)), max(heaps.rows)-1, player)
    elif n_heaps_greater_than_one == 1 and n_heaps_to_one == 0: #[a, 0, 0], a > 1
        # take all objects but 1 from the last non zero heap with more than 1 object
        heaps.nimming(heaps.rows.index(max(heaps.rows)), max(heaps.rows)-1, player)
    elif n_heaps_greater_than_one == 0 and n_heaps_to_one > 0:   #[1, 0, 1, 1]
        # take one object from the first heap
        heaps.nimming(heaps.rows.index(1), 1, player)
    elif n_heaps_greater_than_one == 2 and n_heaps_to_one == 0: #[0, a, b, 0, ..., 0], a, b > 1, a < b
        # reduce the highest heap to the same height as the other
        max_heap = max(heaps.rows)
        max_heap_idx = heaps.rows.index(max_heap)
        min_heap = min(heaps.rows)
        heaps.nimming(max_heap_idx, max_heap - min_heap, player )
    else:  # normal game
        x = nim_sum(heaps.rows)
        y = [nim_sum([x, h]) for _, h in enumerate(heaps.rows)]
        winning_heaps = [i for i, h in enumerate(heaps.rows) if y[i] < h]
        if len(winning_heaps) > 0: # if there's a winning heap
            chosen_heap_idx = random.choice(winning_heaps)
            heaps.nimming(chosen_heap_idx, heaps.rows[chosen_heap_idx]-y[chosen_heap_idx], player)
        else: # take one object from a random heap 
            random_strategy(player, heaps, decrement=True)

In [88]:
# trying nim-sum vs random strategy
heaps = Nim(N_HEAPS)
Alice = Player("Alice", nim_sum_strategy)
Bob = Player("Bob", random_strategy)
winners = match(Alice, Bob, heaps, N_GAMES)
print_match_result(Alice, Bob, winners)



Alice won 86 times
Bob won 14 times


In [89]:
# trying nim-sum vs nim-sum startegy
heaps = Nim(N_HEAPS)
Alice = Player("Alice", nim_sum_strategy)
Bob = Player("Bob", nim_sum_strategy)
winners = match(Alice, Bob, heaps, N_GAMES)
print_match_result(Alice, Bob, winners)


Alice won 69 times
Bob won 31 times


## Task3.2 Evolved Rules

## Task3.3 MinMax

In [90]:
class GameNode:
    def __init__(self, state: list, parent=None, children: list = None):
        self._state = state
        self._parent = parent
        self._children = children
        self._value = 0

    def __hash__(self):
        return hash(bytes(self._state))

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, val):
        self._value = val

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, val):
        self._parent = val

    @property
    def children(self):
        return self._children

    def add_child(self, child):
        self._children.append(child)

    @property
    def state(self):
        return self._state

def heap_distance(node: GameNode):
    # the number of plies to the end of the game using a nim-sum strategy
    # negative if the player loses, positive if the player wins
    this_player = Player("this_player", nim_sum_strategy)
    opponent = Player("opponent", nim_sum_strategy)
    winner = play(this_player, opponent, Nim(rows=copy.deepcopy(node.state)))
    if winner.name == this_player.name:
        return winner.n_plies
    else:   # the opponent won
        return -winner.n_plies


def heuristic(node: GameNode, hash_table: dict):
    # check if the value of the state has been already computed
    h = hash_table.get(node)
    if h is None:
        h = heap_distance(node)  # heuristic formula
        if h == 0:
            h = float('inf')
        else:
            h = 1/h
        hash_table[node] = h  # insert in hash_table for later use
    return h


def minmax(node: GameNode, depth: int, maximising: bool, hash_table: dict):
    if depth == 0:
        if sum(node.state) == 0: # if the node is a terminal state like [0, 0, 0]
            if maximising:
                node.value = float('inf') # i won because the opponent had like [0, 1, 0] and took the last object
            else:
                node.value = -float('inf') # i lost
        else:
            node.value = heuristic(node, hash_table)
        return node.value
    if maximising:
        node.value = -float('inf')
        for c in node.children:
            node.value = max(node.value, minmax(c, depth-1, False, hash_table))
        return node.value
    else:
        node.value = float('inf')
        for c in node.children:
            node.value = min(node.value, minmax(c, depth-1, True, hash_table))
        return node.value


def game_tree(state: list, parent: GameNode, depth: int) -> GameNode:
    this_node = GameNode(state, parent, list())
    if depth > 0:
        for i in range(len(state)):
            # list all the possible new sizes of the heap state[i]
            for j in range(state[i]):
                child_state = copy.deepcopy(state)
                child_state[i] = j
                this_node.add_child(game_tree(child_state, this_node, depth-1))
    return this_node

class MinMaxPlayer(Player):
    def __init__(self, name, strategy, look_ahead):
        super().__init__(name, strategy)
        self._hash_table = {}
        self._look_ahead = look_ahead

    def flush_parameters(self):
        self._hash_table = {}
        super().flush_parameters()

    @property
    def hash_table(self):
        return self._hash_table
    
    @property
    def look_ahead(self):
        return self._look_ahead

def minmax_strategy(player: MinMaxPlayer, heaps: Nim):
    depth = player.look_ahead*2  # depth of the tree is the double of plies look ahead

    # generate game tree, access it through the root
    root = game_tree(heaps.rows, None, depth)

    # apply minmax algorithm, return the heuristic value of the action to be taken
    chosen_value = minmax(root, depth, True, player.hash_table)

    # select actions
    viable_children_idxs = [i for i, c in enumerate(
        root.children) if c.value == chosen_value]
    chosen_child_idx = random.choice(viable_children_idxs)
    chosen_child = root.children[chosen_child_idx]

    # compute the heap idx and the number of object to take
    difference = [i-j for i, j in zip(root.state, chosen_child.state)]
    num_objects = max(difference)
    chosen_heap = difference.index(num_objects)

    # nim the heap
    heaps.nimming(chosen_heap, num_objects, player)


In [93]:
# minmax vs random
heaps = Nim(N_HEAPS)
Alice = MinMaxPlayer("Alice", minmax_strategy, 1)
Bob = Player("Bob", random_strategy)
winners = match(Alice, Bob, heaps, N_GAMES)
print_match_result(Alice, Bob, winners)

Alice won 56 times
Bob won 44 times


In [96]:
# minmax vs minmax
heaps = Nim(N_HEAPS)
Alice = MinMaxPlayer("Alice", minmax_strategy, 1)
Bob = MinMaxPlayer("Bob", minmax_strategy, 1)
winners = match(Alice, Bob, heaps, N_GAMES)
print_match_result(Alice, Bob, winners)

Alice won 49 times
Bob won 51 times


In [99]:
# minmax vs nimsum
heaps = Nim(N_HEAPS)
Alice = MinMaxPlayer("Alice", minmax_strategy, 2)
Bob = Player("Bob", nim_sum_strategy)
winners = match(Alice, Bob, heaps, N_GAMES)
print_match_result(Alice, Bob, winners)

Alice won 0 times
Bob won 100 times
